In [23]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris


warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [11]:
df = pd.read_csv('../data/processed/df_procesado.csv', index_col=0)

In [14]:
df.columns

Index(['Number of Children', 'Smoking Status', 'Physical Activity Level',
       'Employment Status', 'Income', 'Alcohol Consumption', 'Dietary Habits',
       'Sleep Patterns', 'History of Mental Illness',
       'History of Substance Abuse', 'Family History of Depression',
       'Age_Adulthood', 'Age_Eld', 'Age_Young Adulthood', 'Log_Income',
       'Status_Divorced', 'Status_Married', 'Status_Single', 'Status_Widowed',
       'Education_Associate Degree', 'Education_Bachelor's Degree',
       'Education_High School', 'Education_Master's Degree', 'Education_PhD',
       'Target'],
      dtype='object')

## División en train y test

In [15]:
x = df.drop(columns = 'Target')
y = df['Target']

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.2,
                                                    random_state=42)

In [21]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(331014, 24)
(82754, 24)
(331014,)
(82754,)


## Modelo 1: Regresión Logística

In [ ]:
pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ('classifier', LogisticRegression(random_state=42))
])

# Entrenar y evaluar
pipeline_lr.fit(X_train, y_train)
score_lr = pipeline_lr.score(X_test, y_test)
print(f"Regresión Logística - Accuracy: {score_lr:.2f}")


## Modelo 2: Random Forest

In [ ]:
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Entrenar y evaluar
pipeline_rf.fit(X_train, y_train)
score_rf = pipeline_rf.score(X_test, y_test)
print(f"Random Forest - Accuracy: {score_rf:.2f}")


## Modelo 3: XGBoost

In [ ]:
pipeline_xgb = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'))
])

# Entrenar y evaluar
pipeline_xgb.fit(X_train, y_train)
score_xgb = pipeline_xgb.score(X_test, y_test)
print(f"XGBoost - Accuracy: {score_xgb:.2f}")

## Modelo 4: KNN

In [ ]:
pipeline_knn = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ('classifier', KNeighborsClassifier())
])

# Entrenar y evaluar
pipeline_knn.fit(X_train, y_train)
score_knn = pipeline_knn.score(X_test, y_test)
print(f"KNN - Accuracy: {score_knn:.2f}")

## Modelo 5: SVM

In [ ]:
pipeline_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ('classifier', SVC(random_state=42))
])

# Entrenar y evaluar
pipeline_svm.fit(X_train, y_train)
score_svm = pipeline_svm.score(X_test, y_test)
print(f"SVM - Accuracy: {score_svm:.2f}")

Pendiente: Meter a cada modelo un gridsearch, evaluacion y visualizar esas evaluaciones. Importancia de recall